Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Yi Ting Huang,John Enright, Subhash Madineni (GROUP 9)"
COLLABORATORS = ""

---

In [ ]:
def test():
    pass

# Create as many cells as you need BELOW this cell

In [1]:
from IPython.display import display, HTML
import pandas as pd
import sqlite3
from sqlite3 import Error
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
from urllib.request import urlopen
import matplotlib.pyplot as plt
import matplotlib as mpl_dates
from datetime import datetime, timedelta
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA

def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn
##Create Tables
conn = create_connection('Unemployment.db')
sql_statement = "select * from Unemployment where year = 2020 or year =2019;"
df = pd.read_sql_query(sql_statement, conn)
display(df)

conn = create_connection('Unemployment.db')
sql_statement = "select * from income;"
df1 = pd.read_sql_query(sql_statement, conn)
display(df)

#Join Tables
df1['Name of County']= df1['Name of County'].astype(str)
list_lower =[]
for county in df1['Name of County']:
    if 'County' in county.split():
        list_lower.append( county.replace(' County',''))
len(list_lower)
df1 = df1.rename(columns={'':'Median_Income'})
df2 = pd.DataFrame()
df2['County'] = list_lower
df2['Median_Income'] = df1['Median_Income'].tolist()[:62]

main_df = df.merge(df2,on=['County'])
main_df


DatabaseError: Execution failed on sql 'select * from Unemployment where year = 2020 or year =2019;': no such table: Unemployment

In [ ]:
main_df

In [ ]:
list_lower =[]
for county in df1['Name of County']:
    if 'County' in county.split():
        list_lower.append( county.replace(' County',''))
len(list_lower)

In [ ]:
df_2020 = main_df[main_df['Year']==2020]
df_2019 = main_df[main_df['Year']==2019]

In [ ]:
df1 = df1.rename(columns={'':'Median_Income'})
df2 = pd.DataFrame()

In [ ]:
with urlopen('https://cugir.library.cornell.edu/download/file/cugir-007865-geojson.json') as response:
    counties = json.load(response)

In [ ]:
counties

In [ ]:
for i in range(len(counties['features'])) :
    print(counties['features'][i]['properties']['name'])

In [ ]:
inc = main_df['Median_Income'].tolist()
for i in range(len(inc)) :
    inc[i]=inc[i].replace(',','')
main_df['Median_Income'] = inc
main_df['Median_Income'] = main_df['Median_Income'].astype(int)
main_df

In [ ]:
main_df['County'].unique().tolist()

In [ ]:
coun = main_df['County'].unique().tolist()
mean_amount = []
for c in coun :
    mean_amount.append(df_2020[df_2020['County']==c]['Benefit_Amount'].mean())
dd = pd.DataFrame()
dd['County']= coun
dd['Benefit_Amount']=mean_amount

In [ ]:
df_2020[df_2020['County']=='Albany']['Benefit_Amount'].mean()

In [ ]:
# Data Vizulizations

In [ ]:
#Mean beneficiary Amounts 2019 vs 2020

dfpr1 = main_df[main_df['Year']==2019].groupby(['Month'])['Benefit_Amount'].mean()
vals = []
for i in list(range(1,13)) :
    vals.append(dfpr1[i])
vals
fig1 = go.Figure()
fig1.update_layout(title_text = 'Mean Beneficiary Amounts 2019 vs 2020')
fig1.add_trace(
    go.Scatter(x=main_df['Month'], y=main_df[main_df['Year']==2020].groupby(['Month'])['Benefit_Amount'].mean(),line=dict(color='firebrick', width=2), mode='lines',name="2020"))
fig1.add_trace(
    go.Scatter(x=list(range(1,13)), y=vals,line=dict(color='royalblue', width=2), mode='lines',name="2019"))

In [ ]:
#Choropleth map showing the median household income(2018) by county

fig = px.choropleth(main_df, geojson=counties, locations='County', color='Median_Income',
                           range_color=(0, main_df['Median_Income'].max()),
                           scope="usa",featureidkey="properties.name",
                    
                           labels={'Median_Income':'Median Income per County'}
                          )

fig.show()

In [ ]:
#Benefit amnount paid out in 2019

coun = main_df['County'].unique().tolist()
mean_amount = []
for c in coun :
    mean_amount.append(df_2019[df_2019['County']==c]['Benefit_Amount'].mean())
dd = pd.DataFrame()
dd['County']= coun
dd['Benefit_Amount']=mean_amount

fig = px.choropleth(dd, geojson=counties, locations='County', color='Benefit_Amount',
                           range_color=(dd['Benefit_Amount'].min(), dd['Benefit_Amount'].max()),
                           scope="usa",featureidkey="properties.name",
                    
                           labels={'Benefit_Amount':'Benefit Amount per County'}
                          )

fig.show()

In [ ]:
#Benefit amount paid out by county in 2020

coun = main_df['County'].unique().tolist()
mean_amount = []
for c in coun :
    mean_amount.append(df_2020[df_2020['County']==c]['Benefit_Amount'].mean())
dd = pd.DataFrame()
dd['County']= coun
dd['Benefit_Amount']=mean_amount

fig = px.choropleth(dd, geojson=counties, locations='County', color='Benefit_Amount',
                           range_color=(dd['Benefit_Amount'].min(), dd['Benefit_Amount'].max()),
                           scope="usa",featureidkey="properties.name",
                    
                           labels={'Benefit_Amount':'Benefit Amount per County'}
                          )

fig.show()

In [ ]:
dfpr1 = main_df[main_df['Year']==2019].groupby(['Month'])['Benefit_Amount'].mean()
vals = []
for i in list(range(1,13)) :
    vals.append(dfpr1[i])
vals
fig1 = go.Figure()
fig1.update_layout(title_text = 'Mean Beneficiary Amounts 2019 vs 2020')
fig1.add_trace(
    go.Scatter(x=main_df['Month'], y=main_df[main_df['Year']==2020].groupby(['Month'])['Benefit_Amount'].mean(),line=dict(color='firebrick', width=2), mode='lines',name="2020"))
fig1.add_trace(
    go.Scatter(x=list(range(1,13)), y=vals,line=dict(color='royalblue', width=2), mode='lines',name="2019"))

In [ ]:
# Coefficent Relationships

In [ ]:
## converting income to int
inc = df_2020['Median_Income'].tolist()
for i in range(len(inc)) :
    inc[i]=inc[i].replace(',','')
df_2020['Median_Income'] = inc
df_2020['Median_Income'] = df_2020['Median_Income'].astype(int)

In [ ]:
inc = df_2019['Median_Income'].tolist()
for i in range(len(inc)) :
    inc[i]=inc[i].replace(',','')
df_2019['Median_Income'] = inc
df_2019['Median_Income'] = df_2019['Median_Income'].astype(int)

In [ ]:
cor = df_2020.corr()
cor.style.background_gradient(cmap='BrBG')

In [ ]:
cor2 = df_2019.corr()
cor2.style.background_gradient(cmap = 'BrBG')

In [ ]:
# Principle Component Analysis 

In [ ]:
x = df_2020[['Median_Income','Beneficiaries']]

In [ ]:
## Scaling the data for PCA
from sklearn.preprocessing import StandardScaler
x = StandardScaler().fit_transform(x)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
principalComponents = pca.fit_transform(x)

In [ ]:
principalComponents

In [ ]:
target = df_2020['Benefit_Amount']
target.max()

In [ ]:
## splitting the benefit amounts into 6 categories based on range of payments
target.ravel()
bins = pd.cut(target, 6, labels = ['very_low', 'low', 'medium', 'medium_high', 'high', 'very_high'])
bins

In [ ]:
## combining scaled pc's with now categorical benefit_amount 
finalDf = pd.concat([principalDf, bins], axis = 1)
finalDf

In [ ]:
## Plotting the PCA Analysis
import matplotlib.pyplot as plt

In [ ]:
plt.style.use('ggplot')
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = ['very_low', 'low', 'medium', 'medium_high', 'high', 'very_high']
colors = ['r', 'g', 'b','y', 'c', 'm' ]
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['Benefit_Amount'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid(linestyle = '-', color = 'white')

In [ ]:
## component 1 describes 61% of varaince while PC2 explains ~ 39%
## where component 1 is median_income and 2 is # beneeficiaries
pca.explained_variance_ratio_

In [ ]:
#Random Forest Regression

In [ ]:
## First, have to encode the variables, drop beneficiaries because of its high correlation with benefit_amount
## Beneficiaries used later for comparison to model predictions
features = pd.get_dummies(df_2020.drop('Beneficiaries', axis = 1))
features.head(6)

In [ ]:
## separating the benefit_amount
labels = np.array(features['Benefit_Amount'])

In [ ]:
features= features.drop('Benefit_Amount', axis = 1)

In [ ]:
feature_list = list(features.columns)

In [ ]:
## splitting to test and train
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [ ]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

In [ ]:
## training the model
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(train_features, train_labels)

In [ ]:
## predicting the test set
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error
print('Mean Absolute Error:', round(np.mean(errors), 2), 'dollars.', "In comparison to the max 430,000,000 dollars.")

In [ ]:
## Listing the variables and their relative importance to predicting benefit_amount
importances = list(rf.feature_importances_)
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:24} Importance: {}'.format(*pair)) for pair in feature_importances];

In [ ]:
## getting the non-zero features and variables for visualizations
fm = feature_importances[0:11]

In [ ]:
imp = []
for a,b in fm:
    imp.append(b)

In [ ]:
## Importance Plot For Benefit_Amount
plt.style.use('ggplot')
x_values = list(range(len(imp)))
plt.bar(x_values, imp, orientation = 'vertical', color=(0.2,0.4,0.6))
plt.xticks(x_values,fm, rotation='vertical')
plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances')
plt.figure(figsize = (8,8))
plt.tight_layout()
;

In [ ]:
sql_statement = '''select year,
                    sum(Benefit_Amount) as Benifit_Amount
                    from Unemployment 
                    group by year
                    limit 20;'''
df = pd.read_sql_query(sql_statement, conn)
#display(df)

x = df['Year']
y = df['Benifit_Amount']
plt.plot(x,y)
plt.ylabel('Benifit Amount')
plt.xlabel(' Year')
#plt.legend([''])
plt.show()

In [ ]:
sql_statement = '''select month,
                    sum(Beneficiaries) as Beneficiaries
                    from unemployment 
                    where year = 2020
                    group by month  ;'''

sql_statement1 = '''select month,
                    sum(Beneficiaries) as Beneficiaries
                    from unemployment 
                    where year = 2019
                    group by month  ;'''
sql_statement2 = '''select month,
                    sum(Beneficiaries) as Beneficiaries
                    from unemployment 
                    where year = 2009
                    group by month  ;'''
sql_statement3 = '''select month,
                    sum(Beneficiaries) as Beneficiaries
                    from unemployment 
                    where year = 2008
                    group by month  ;'''
df_2020 = pd.read_sql_query(sql_statement, conn)
df_2019 = pd.read_sql_query(sql_statement1, conn)
df_2008 = pd.read_sql_query(sql_statement2, conn)
df_2009 = pd.read_sql_query(sql_statement3, conn)

plt.plot(df_2020['Month'],df_2020['Beneficiaries'])
plt.plot(df_2019['Month'],df_2019['Beneficiaries'])
plt.plot(df_2008['Month'],df_2008['Beneficiaries'])
plt.plot(df_2009['Month'],df_2009['Beneficiaries'])


plt.ylabel('Unemployment payout claiming People')
plt.xlabel(' Month')
plt.legend([2020,2009,2008,2019])
plt.show()

In [ ]:
from datetime import datetime
from matplotlib.pylab import rcParams
sql_statement = '''select Year||'-'|| month as Date,  
                    sum(Beneficiaries) as Beneficiaries 
                    from unemployment  
                     group by year,month 
                     order by year,month
                     '''

df = pd.read_sql_query(sql_statement, conn)
df.drop(238, axis = 0, inplace = True)
display(df.tail())

df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format= True)
indexedDataset =df.set_index(['Date'])
plt.xlabel("Date")
plt.ylabel("Number of Beneficiaries")
plt.plot(indexedDataset)
plt.show()

In [ ]:
# H0 Data is NON-Stationary
# H1 Data is stationary
from statsmodels.tsa.stattools import adfuller
def adfuller_test(indexedDataset):
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(indexedDataset['Beneficiaries'], autolag = 'AIC')

    dfoutput = pd.Series(dftest[0:4], index = ['Test Statistic', 'p-value', '#Lags Used','No of Observations Used'])
    # for key,value in dftest[4].items():
    #     dfoutput['Critical Value (%s)' %key] = value
    display(dfoutput)
    if(dfoutput[1] <= 0.05):
        display('STRONG Evedience against Null hypotheis, reject Null hypothesis- DATA IS STATIONARY')
    else:
        display('WEAK Evedience against Null hypotheis,Time-series has a unit Root, Indicating -- DATA IS STATIONARY')

In [ ]:
#Perform AD-Fuller test:
adfuller_test(indexedDataset)

In [ ]:
datasetDiffShifting = indexedDataset - indexedDataset.shift(6)
plt.plot(datasetDiffShifting)
plt.show()
#display(datasetDiffShifting)

In [ ]:
## Again test dickey fuller test
adfuller_test(datasetDiffShifting.dropna())

In [ ]:
datasetDiffShifting.dropna(inplace = True)
#ACF and PACF plots:

from statsmodels.tsa.stattools import acf,pacf

lag_acf = acf(datasetDiffShifting, nlags = 20)
lag_pacf = pacf(datasetDiffShifting, nlags = 20, method = 'ols')


#plot ACF
plt.subplot(121)
plt.plot(lag_acf)
plt.axhline(y=0,linestyle = '--', color = 'gray')
plt.axhline(y=-1.96/np.sqrt(len(datasetDiffShifting)),linestyle = '--', color = 'gray')
plt.axhline(y=-1.96/np.sqrt(len(datasetDiffShifting)),linestyle = '--', color = 'gray')
plt.title("Autocorrelation Functions")

#plot PACF
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0,linestyle = '--', color = 'gray')
plt.axhline(y=-1.96/np.sqrt(len(datasetLogDiffShifting)),linestyle = '--', color = 'gray')
plt.axhline(y=-1.96/np.sqrt(len(datasetLogDiffShifting)),linestyle = '--', color = 'gray')
plt.title("Autocorrelation Functions")

plt.tight_layout()

In [ ]:
#ARIMA MODEL
model = ARIMA(indexedDataset, order=(2,1,2))
results_ARIMA = model.fit(disp = -1)
# plt.plot(datasetDiffShifting)
# plt.plot(results_ARIMA.fittedvalues, color = 'red')
# plt.title('RSS: %.4f'% sum(results_ARIMA.fittedvalues-datasetDiffShifting['Beneficiaries'])**2)
# print('Plotting AR model')

results_ARIMA.plot_predict(1,298) 

#238 past rows + 60( next 5 years = 12months *5 years)